# Goodreads Scraping Tool

In [ ]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import csv


In [ ]:
urls = []
headers= {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Mobile Safari/537.36'}
#enter your login credentials below
login_data={'user[email]':'Your Email', 'user[password]': 'Your Password','authenticity_token':'token','next':'Sign in','n':'number'}
with requests.Session() as s:
    url='https://www.goodreads.com/user/sign_in?returnurl=%2Fshelf%2Fshow%2Fbestsellers'
    r = s.get(url,headers=headers)
    soup = BeautifulSoup(r.content,'html5lib')
    login_data['authenticity_token']=soup.find('input', attrs={'name': 'authenticity_token'})['value']
    login_data['n']=soup.find('input', attrs={'name': 'n'})['value']
    r= s.post(url,data=login_data,headers=headers)
    for pn in range(1,5):
        baseUrl = 'https://www.goodreads.com'
        path = 'shelf/show/bestsellers?page='+str(pn) #change path if you want to get books from another list
        page = s.get(baseUrl + path).text
        soup = BeautifulSoup(page, "html.parser")

        #for link in soup.findAll('a',attrs={'class':"elementList",'class':"leftAlignedImage"}):
        for link in soup.findAll('a',attrs={'class':"bookTitle",'itemprop':'url'}):
            if link['href'].startswith('/author/show/'):
                pass
            else:
                u=baseUrl+link['href']
                urls.append(u)
   

In [ ]:
books = []
title=""
def scrape_title(url):
    data = requests.get(url).text
    idtitle="bookTitle"
    idseries="bookSeries"
    soup = BeautifulSoup(data, "html.parser")
    title = (soup.select("#" + idtitle)[0].get_text().strip()+" "+soup.select("#" + idseries)[0].get_text().strip()).strip()    #Note: I have used Index to select the first element in list.
    books.append(title)
    return title
author=""
def scrape_author(url):
    data = requests.get(url).text
    id="authorName"
    soup = BeautifulSoup(data, "html.parser")
    for a in soup.findAll(attrs={'class':"authorName"})[0]:
        author=a.get_text()
    return author
genre=""
def scrape_genre(url):
    genre=""
    data = requests.get(url).text
    id="bookPageGenreLink"
    soup = BeautifulSoup(data, "html.parser")
    if len(soup.findAll(attrs={'class':id}))>0:
        for g in soup.findAll(attrs={'class':id})[0]:
            genre=g
    return genre
# Scrapes description data from goodreads.com
def scrape_description(url):
    data = requests.get(url).text
    id="description"
    soup = BeautifulSoup(data, "html.parser")
    if soup.find('div',attrs={'id':id})!=None:
        desc = soup.select("#" + id)[0].text
    else:
        desc=""    #Note: I have used Index to select the first element in list. 
    return desc
def scrape_rating(url):
    data = requests.get(url).text
    id="ratingValue"
    soup = BeautifulSoup(data, "html.parser")
    rating= float(soup.find('span',attrs={'itemprop':id}).get_text())
    return rating
def scrape_nbratings(url):
    data = requests.get(url).text
    id="ratingCount"
    soup = BeautifulSoup(data, "html.parser")
    nbratings= int(soup.find('meta',attrs={'itemprop':id}).get('content'))
    
    return nbratings
def scrape_nbpages(url):
    nbpages=None
    data = requests.get(url).text
    id="numberOfPages"
    soup = BeautifulSoup(data, "html.parser")
    if soup.find('span',attrs={'itemprop':id})!=None:
        nbpages= int(soup.find('span',attrs={'itemprop':id}).get_text().split()[0])
    return nbpages
def scrape_format(url):
    bookformat="Paperback"
    data = requests.get(url).text
    id="bookFormat"
    soup = BeautifulSoup(data, "html.parser")
    if soup.find('span',attrs={'itemprop':id})!=None:
        bookformat= soup.find('span',attrs={'itemprop':id}).get_text()
    return bookformat

In [ ]:
#dict={}
book_details=[]
book_list=[]
class book:
    def __init__(self,title,author,genre,description,rating,nbratings,nbpages):
        self.title=title
        self.author=author
        self.genre=genre
        self.description=description
        self.rating=rating
        self.nbratings=nbratings
        self.nbpages=nbpages
    def set_title(self,title):
        self.title=title
    def get_title(self):
        return self.title
    def set_author(self,author):
        self.author=author
    def get_author(self):
        return self.author
    def set_genre(self,genre):
        self.genre=genre
    def get_genre(self):
        return self.genre
    def set_description(self,description):
        self.description=description
    def get_description(self):
        return self.description
    def set_rating(self,rating):
        self.rating=rating
    def get_rating(self):
        return self.rating
    def set_nbratings(self,nbratings):
        self.nbratings=nbratings
    def get_nbratings(self):
        return self.nbratings
    def set_nbpages(self,nbpages):
        self.nbpages=nbpages
    def get_nbpages(self):
        return self.nbpages


In [ ]:
for url in urls:
    t=scrape_title(url)
    print(t)
    a=scrape_author(url)
    g=scrape_genre(url)
    d=scrape_description(url)
    r=scrape_rating(url)
    nbr=scrape_nbratings(url)
    nbp=scrape_nbpages(url)
    f=scrape_format(url)
    book_details=[t,a,g,d,r,nbr,nbp,f]
    book_list.append(book_details)
    
    #dict[b.get_title()]={'Author':b.get_author(),'Genre':b.get_genre(),'Description':b.get_description(),'Rating':b.get_rating(),'Number of Ratings':b.get_nbratings(),'Number of Pages':b.get_nbpages()}
    

In [ ]:
#Print books to csv
import csv
with open('bookdata.csv', 'a', newline='',encoding="utf-8") as csvfile:
    fieldnames = ['Title','Author','Genre','Description','Rating','NB Ratings','NB Pages','Format']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    #writer.writeheader()
    for i in range(len(book_list)):
        writer.writerow({'Title':book_list[i][0], 'Author':book_list[i][1], 'Genre':book_list[i][2],'Description':book_list[i][3],'Rating':book_list[i][4],'NB Ratings':book_list[i][5],'NB Pages':book_list[i][6],'Format':book_list[i][7]})